# Write a grid file

Either from scratch given a certain number of binaries, or based off the output from an already existing grid

In [22]:
import numpy as np
import random
import math
import os 
import pandas as pd
from astropy.table import Table
import h5py as h5

N_binaries          = int(1e6)
home_dir            = os.path.expanduser("~") 
bse_grid_loc        = home_dir + "/Winds2/code/run_data/" #"/Winds2/code/GridSubmitCOMPAS/masterfolder/"
bse_grid_filename   = 'BSE_grid_mass_sep_kick.txt'


In [19]:
# Draw values from a power law (using inverse transform sampling method)
def draw_power_law(N = 10, gamma = -2.3, m_min = 1, m_max = 150):
    # For gamma = 0, its just a uniform dist
    if gamma == 0:
        return np.random.uniform(m_min, m_max, N)
    
    # For 1/x the integral (CDF) is ln(x)
    elif gamma == -1:
        y = np.random.uniform(0, 1, N)# will be used to draw from the inverse CDF
        return np.exp(y * (np.log(m_max) - np.log(m_min)) + np.log(m_min))
    
    # For the general powerlaw case
    else:
        y = np.random.uniform(0, 1, N)
        return ((m_max**(gamma+1) - m_min**(gamma+1))*y + m_min**(gamma+1))**(1/(gamma+1))

# Draw primary masses 
### between 5 and 150 Msun from a Kroupa IMF
Actually I am just going to draw stuff above 5Msun so, essentially a saltpeter, or just a powerlaw

In [16]:
# Since I'm only drawing stuff above 5 Msun, I wont really care and just assume essentially a saltpeter IMF
M_primary = draw_power_law(N = N_binaries, gamma = -2.3, m_min = 5, m_max = 150)
print(M_primary)

[ 6.33962237  5.50299702  9.75787329 ...  9.3611681   5.36003545
 23.01928173]


# Draw secondary masses

## by drawing q = M2/M1 from a flat distribution between [0.01 - 1] Msun

In [17]:
# Draw N_binaries in q from a flat distribution
Q = np.random.uniform(0.01, 1, N_binaries) # will be used to draw from the inverse CDF

# get M_secondary values 
M_secondary = M_primary*Q
print('M_secondary', M_secondary) # is in Msun

# Check that none of the M2 are smaller than --minimum-secondary-mass = 0.1 (default)
# Assume M_primary is already defined
while np.any(M_secondary < 0.1):
    mask = M_secondary < 0.1
    Q[mask] = np.random.uniform(low=0.1/M_primary[mask], high=1, size=np.sum(mask))
    M_secondary[mask] = Q[mask] * M_primary[mask]




M_secondary [ 4.664862    1.35255271  7.95282971 ...  3.8574218   0.94806464
 21.78839725]


# Draw initial semi major axis

### From a flat-in-log distribution between 0.01 - 1000 AU

$$
p(a_i) = \frac{1}{a_i}
$$

In [21]:
SMA_init = draw_power_law(N = N_binaries, gamma = -1, m_min = 0.01, m_max = 1000)
print('SMA_init', SMA_init)


SMA_init [1.91433634e+01 2.69965957e+02 1.29305243e-02 ... 2.73336797e+02
 5.88040154e+01 3.21011975e+00]


# Drawing Kicks

We need to draw kick magnitudes and orientations

### kick magnitudes:
By default, COMPAS assumes for BHs that kick magnitudes are drawn from a maxwellian (e.g., Hobbs et al., 2005),  with a sigma = 265.0 km/s
_However_  we don't actually need to draw this ourselves! the parameters **--kick-magnitude-random-1**, and     **--kick-magnitude-random-2**, will cause a unique draw from this magnitude distribution for you!
Default = Random number drawn uniformly from [0.0, 1.0)

### Kick orientations:
By default, COMPAS draws natal kick directions from an ISOTROPIC distribution. 
i.e.,

**--kick-direction ISOTROPIC** 

This means that we want to set 
1. **--kick-theta-1:**   angle between the orbital plane and the 'z' axis of the supernova vector for the primary star should it undergo a supernova event [radians]
    
    Generate cos(θ) (cosine of θ) randomly from a uniform distribution between -1 and 1.
    Calculate θ (theta) using θ = arccos(cos(θ)), where arccos is the inverse cosine function.

    
2. **--kick-phi-1:**   angle between 'x' and 'y', both in the orbital plane of the supernova vector, for the primary star should it undergo a supernova event [radians]

    Generate phi randomly from a uniform distribution between 0 and 2π.
        
3. **--kick-mean-anomaly-1:**  The mean anomaly at the instant of the supernova for the primary star of a binary system when evolving in BSE mode, should it undergo a supernova event. 
    Default = Random number drawn uniformly from [0.0, 2pi)
        



In [24]:
# Kick magnitude numbers
Kick_magnitude_random1 = np.random.uniform(0.0, 1, N_binaries) 
Kick_magnitude_random2 = np.random.uniform(0.0, 1, N_binaries) 

# Calculate theta using theta = arccos(cos(theta))
theta1 = np.arccos(np.random.uniform(-1.0, 1.0, N_binaries))
theta2 = np.arccos(np.random.uniform(-1.0, 1.0, N_binaries))

# kick phi Generate random number between 0 and 2 pi
phi1 = np.random.uniform(0.0, 2 * np.pi, N_binaries)
phi2 = np.random.uniform(0.0, 2 * np.pi, N_binaries)

# Mean anomaly: Generate N_binaries random numbers between 0 and 2 pi
mean_anomaly1 = np.random.uniform(0.0, 2 * np.pi, N_binaries)
mean_anomaly2 = np.random.uniform(0.0, 2 * np.pi, N_binaries)


# Write this to a grid file

In [25]:
keys = ['--random-seed', '--initial-mass-1', '--initial-mass-2', '--semi-major-axis',
        '--kick-magnitude-random-1', '--kick-phi-1', '--kick-theta-1', '--kick-mean-anomaly-1',
        '--kick-magnitude-random-2', '--kick-phi-2', '--kick-theta-2', '--kick-mean-anomaly-2']

# Open a file to write the data to
with open(bse_grid_loc + bse_grid_filename, 'w') as f:
    for i in range(N_binaries):
        values = [str(i), M_primary[i], M_secondary[i], SMA_init[i],
                  Kick_magnitude_random1[i], phi1[i], theta1[i], mean_anomaly1[i],
                  Kick_magnitude_random2[i], phi2[i], theta2[i], mean_anomaly2[i]]

        flags = ' '.join(f'{key} {value}' for key, value in zip(keys, values))

        # Write the flags to the file, each set of flags on a new line
        f.write(flags + '\n')
            

In [32]:
# Next we will write these kick magnitudes to BSE_grid_mass_sep_kick

# Read the kick magnitudes from the text file
kick_magnitudes = pd.read_csv(bse_grid_loc + 'kick_magnitudes.txt', sep='\t')

# Open the BSE_grid file
with open(bse_grid_loc+'BSE_grid_mass_sep_kick.txt', 'r') as file:
    lines = file.readlines()


# Iterate over the lines
for i, line in enumerate(lines):
    words = line.split()
    words[8] = "--kick-magnitude-1"
    words[9] = str(kick_magnitudes['Drawn_Kick_Magnitudes_1'][i])
    words[16] = "--kick-magnitude-2"
    words[17] = str(kick_magnitudes['Drawn_Kick_Magnitudes_2'][i])
    lines[i] = ' '.join(words) + '\n'


# Write the modified lines back to a new file
with open(bse_grid_loc+'BSE_grid_mass_sep_kick_copy.txt', 'w') as file:
    file.writelines(lines)


***

# Other techniques

***
# Grid file containing only metallicities

For your fiducial run, you just want to run COMPAS with fiducial flags, but drawing metallicities that are distributed uniformly in log


In [33]:
#################################################################
## 
##   Write a grid file to use
##
#################################################################
def write_grid_file(N = 10, grid_file_name = 'BSE_grid.txt' ):
    """
    N is number of lines
    """

    # Define the minimum and maximum values for Z
    Z_min = 1e-4
    Z_max = 0.03

    # Set a random seed for reproducibility
    random.seed(42)

    # Open the output file for writing
    with open(grid_file_name, 'w') as f:
        # Write each Z value to a separate line in the file, with a random seed integer
        for i in range(N):
            Z = 10 ** random.uniform(math.log10(Z_min), math.log10(Z_max))
            f.write('--random-seed {} --metallicity {}\n'.format(i, Z))


write_grid_file(N = N_binaries, grid_file_name = './masterfolder/BSE_grid.txt' )

# Write a grid file based on the system params of an existing simulation

By reading the 'BSE_System_Parameters', make a grid file to run with compas 
at least containing the parameters as set in Grid_dict 

The grid will be quite large (~1 Gb for 10^6 binaries) so this takes about a minute


In [40]:

# Define the path to the HDF5 file
root_out_dir = "/n/holystore01/LABS/hernquist_lab/Users/lvanson/CompasOutput/v02.35.02/FiducialN1e6/"
filename     = "MainRun/COMPAS_Output.h5"
filepath     = root_out_dir + filename

## WARNING ##
"""
BSE_System_Parameters will only contain all the kick information if the simulation was run 
using a grid, and with the flag --add-options-to-sysparms = 'GRID'
"""
# Define the dictionary of keys and command line arguments
Grid_dict = {'SEED':'--random-seed', 'Mass@ZAMS(1)': '--initial-mass-1', 'Mass@ZAMS(2)':'--initial-mass-2',
             'SemiMajorAxis@ZAMS':'--semi-major-axis','Metallicity@ZAMS(1)':'--metallicity',\
             'Kick_Magnitude_Random(1)':'--kick-magnitude-random-1', 'Kick_Phi(1)':'--kick-phi-1','Kick_Theta(1)':'--kick-theta-1', 'Kick_Mean_Anomaly(1)':'--kick-mean-anomaly-1',\
             'Kick_Magnitude_Random(2)':'--kick-magnitude-random-2', 'Kick_Phi(1)':'--kick-phi-2','Kick_Theta(2)':'--kick-theta-2', 'Kick_Mean_Anomaly(2)':'--kick-mean-anomaly-2'}
             
# Open the HDF5 file in read mode
with h5py.File(filepath, 'r') as data:

    # Get the number of rows in the data
    n_rows = len(data['BSE_System_Parameters']['SEED'])
    
    # Initialize a NumPy array to hold the data
    data_array = np.zeros((n_rows, len(Grid_dict)), dtype=np.float64)
        
    # Loop over the keys in the Grid_dict dictionary
    for i, key in enumerate(Grid_dict.keys()):
        
        # Get the data from the HDF5 file and store it in the data array
        data_key = 'BSE_System_Parameters/' + key
        if key == 'SEED':  # Convert the SEED value to an integer
            data_array[:, i] = data[data_key][()].astype(int)
        else:
            data_array[:, i] = data[data_key][()]  
    
    # Format the data as command line arguments
    data_list = []
    for i in range(n_rows):
        row_data = data_array[i]
        row_list = ['{} {:d}'.format(cmd_arg, int(value)) if cmd_arg == '--random-seed' else '{} {}'.format(cmd_arg, value) for cmd_arg, value in zip(Grid_dict.values(), row_data)]
        data_line = ' '.join(row_list) + '\n'
        data_list.append(data_line)


    # Write the data to the output file
    with open('BSE_grid_N1e6.txt', 'w') as outfile:
        
        # Write the data to the output file
        outfile.writelines(data_list)


In [ ]:
############################################
##  WRITE SEEDS TO GRID RUN FILE
############################################
def write_BSE_grid(SYS, seed_list, save_loc = './',  BSE_grid_name = "BSE_grid.txt"):
    """ Function to write a BSE grid file for your systems of initerest. 
    SYS       = astropy table version of your SystemParameters
    seed_list = the Seeds of interest
    
    """
    Grid_keys = ['SEED','Mass@ZAMS(1)', 'Mass@ZAMS(2)', 'SemiMajorAxis@ZAMS','Metallicity@ZAMS(1)',\
            'SN_Kick_Magnitude_Random_Number(1)', 'SN_Kick_Magnitude_Random_Number(2)',\
#            'SN_Kick_Theta(1)', 'SN_Kick_Theta(2)', 'SN_Kick_Phi(1)', 'SN_Kick_Phi(2)',\
#            'SN_Kick_Mean_Anomaly(1)', 'SN_Kick_Mean_Anomaly(2)']
                ]
    Grid_dict = {'SEED':'--random-seed', 'Mass@ZAMS(1)': '--initial-mass-1', 'Mass@ZAMS(2)':'--initial-mass-2',\
                 'SemiMajorAxis@ZAMS':'--semi-major-axis','Metallicity@ZAMS(1)':'--metallicity',\
            'SN_Kick_Magnitude_Random_Number(1)':'--kick-magnitude-random-1', 'SN_Kick_Magnitude_Random_Number(2)':'--kick-magnitude-random-2',\
#            'SN_Kick_Theta(1)':'--kick-theta-1', 'SN_Kick_Theta(2)':'--kick-theta-2', 'SN_Kick_Phi(1)':'--kick-phi-1', 'SN_Kick_Phi(2)':'--kick-phi-2',\
#            'SN_Kick_Mean_Anomaly(1)':'--kick-mean-anomaly-1', 'SN_Kick_Mean_Anomaly(2)':'--kick-mean-anomaly-2'}
                }
    ##################################
    print('seed_list', seed_list)
    SYS_DCO_seeds_bool = np.in1d(SYS['SEED'][()], seed_list) #Bool to point SYS to DCO
    ##################################
    # Make a reduced table that only consists of your systems of interest
    Sub_SYS = SYS[SYS_DCO_seeds_bool]
    # Make a new file 
#     outF = open(save_loc + str(sys['SEED'])+BSE_grid_name , "w")
    outF = open(save_loc + BSE_grid_name , "w")
    # Loop over every system in seed_list
    for sys in Sub_SYS:
        print(save_loc + str(sys['SEED'])+BSE_grid_name)
        grid_line = ''
        # Append all the necessary keys
        for key in Grid_keys:
            grid_line += ' '+Grid_dict[key]+' '+str(sys[key])
        #print('grid_line',grid_line)
        outF.write(grid_line)
        outF.write("\n")
    outF.close()
